# Tool Calling Functionality LLama

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Get API keys

In [2]:
from utils import get_tavily_api_key
TAVILY_API_KEY = get_tavily_api_key()

## Load helper functions

In [3]:
from utils import llama31
from utils import cprint
import json

## Define tool system prompt

In [4]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
print(formatted_date)

13 October 2024


In [6]:
tool_system_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""
tool_system_prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nEnvironment: ipython\nTools: brave_search, wolfram_alpha\nCutting Knowledge Date: December 2023\nToday Date: 13 October 2024\n'

## The brave_search built-in tool

In [8]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nEnvironment: ipython\nTools: brave_search, wolfram_alpha\nCutting Knowledge Date: December 2023\nToday Date: 13 October 2024\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the current weather in Menlo Park, California?\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

In [9]:
response = llama31(prompt)
print(response)



brave_search.call(query="Menlo Park California weather")


In [11]:
no_tool_call_prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the population of California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
no_tool_call_prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nEnvironment: ipython\nTools: brave_search, wolfram_alpha\nCutting Knowledge Date: December 2023\nToday Date: 13 October 2024\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the population of California?\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

In [12]:
no_tool_call_response = llama31(no_tool_call_prompt)
print(no_tool_call_response)



As of the 2020 United States Census, the population of California is approximately 39.5 million. However, please note that this information may have changed slightly since the last census in 2020.


### Calling the search API

In [15]:
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

result = tavily_client.search("current weather in Lahore with date and time, Pakistan")
cprint(result)

{
    "query": "current weather in Lahore with date and time, Pakistan",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "title": "Weather in Lahore, Pakistan",
            "url": "https://www.weatherapi.com/",
            "content": "{'location': {'name': 'Lahore', 'region': 'Punjab', 'country': 'Pakistan', 'lat': 31.5497, 'lon': 74.3436, 'tz_id': 'Asia/Karachi', 'localtime_epoch': 1728842974, 'localtime': '2024-10-13 23:09'}, 'current': {'last_updated_epoch': 1728842400, 'last_updated': '2024-10-13 23:00', 'temp_c': 23.2, 'temp_f': 73.8, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 104, 'wind_dir': 'ESE', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 73, 'cloud': 0, 'feelslike_c': 23.7, 'feelslike_f': 74.7, 'windchill_c': 24.9, 'windchill_f': 76.8, 'heatin

### Reprompting Llama with search tool response

In [16]:
search_result = result["results"][0]["content"]
print(search_result)

{'location': {'name': 'Lahore', 'region': 'Punjab', 'country': 'Pakistan', 'lat': 31.5497, 'lon': 74.3436, 'tz_id': 'Asia/Karachi', 'localtime_epoch': 1728842974, 'localtime': '2024-10-13 23:09'}, 'current': {'last_updated_epoch': 1728842400, 'last_updated': '2024-10-13 23:00', 'temp_c': 23.2, 'temp_f': 73.8, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 7.2, 'wind_kph': 11.5, 'wind_degree': 104, 'wind_dir': 'ESE', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 73, 'cloud': 0, 'feelslike_c': 23.7, 'feelslike_f': 74.7, 'windchill_c': 24.9, 'windchill_f': 76.8, 'heatindex_c': 24.6, 'heatindex_f': 76.3, 'dewpoint_c': 6.0, 'dewpoint_f': 42.8, 'vis_km': 2.0, 'vis_miles': 1.0, 'uv': 0.0, 'gust_mph': 15.0, 'gust_kph': 24.2}}


In [21]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Lahore, Pakistan?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>{response}<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{search_result}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 13 October 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Lahore, Pakistan?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>The current weather in Menlo Park, California is not explicitly stated in the search results. However, based on the search results for Lahore, Pakistan, the current weather conditions are:

- Temperature: 23.2°C (73.8°F)
- Condition: Mist
- Wind: 7.2 mph (11.5 km/h) from ESE
- Humidity: 73%

Please note that the search results are for Lahore, Pakistan, and not Menlo Park, California. To get the current weather in Menlo Park, California, I would recommend searching for a weather website or API that provides current weather conditions for specific locations.<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{'location': {'name': 'Laho

In [22]:
response = llama31(prompt)
print(response)

The current weather in Lahore, Pakistan is:

- Temperature: 23.2°C (73.8°F)
- Condition: Mist
- Wind: 7.2 mph (11.5 km/h) from ESE
- Humidity: 73%

Please note that these conditions are subject to change and may not reflect the current weather at the time of your query.


## Using the higher-level message

In [23]:
system_prompt_content = f"""
Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [28]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",   "content": "What is the current weather in Rahim Yar khan, Pakistan?"}
  ]

In [25]:
response = llama31(messages)
print(response)

brave_search.call(query="Menlo Park, California weather")


In [33]:
messages = [
    {"role": "system",     "content":  system_prompt_content},
    {"role": "user",       "content": "What is the current weather in Rahim Yar khan, Pakistan?"},
    {"role": "assistant",  "content": response},
    {"role": "ipython",    "content": search_result}
  ]

In [34]:
response = llama31(messages)
print(response)

The current weather in Rahim Yar Khan, Pakistan is misty with a temperature of 23.2°C (73.8°F) and a wind speed of 11.5 km/h (7.2 mph) from the east-southeast. The humidity is 73% and the pressure is 1012.0 mb. There is no precipitation expected.


## The Wolfram Alpha tool

In [35]:
math_problem = "Can you help me solve this equation: x^3 - 2x^2 - x + 2 = 0?"

In [36]:
messages = [
    {"role": "system",  "content": system_prompt_content},
    {"role": "user",    "content": math_problem}
]

In [37]:
response = llama31(messages)
print(response)

wolfram_alpha.call(query="solve x^3 - 2x^2 - x + 2 = 0")


### Calling the Wolfram Alpha tool

In [38]:
from utils import wolfram_alpha
tool_result = wolfram_alpha("solve x^3 - 2x^2 - x + 2 = 0")
print(tool_result)

x = ± 1
x = 2


### Checking the result

In [40]:
from sympy import symbols, Eq, solve

x = symbols('x')                           # Define the variable
equation = Eq(x**3 - 2*x**2 - 1*x + 2, 0) # Define the equation
solution = solve(equation, x)              # Solve the equation

print(solution)

[-1, 1, 2]


### Reprompting Llama with Wolfram Alpha tool response

In [41]:
messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user",      "content": math_problem},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": tool_result}
]

In [43]:
response = llama31(messages)
print(response)

The solutions to the equation x^3 - 2x^2 - x + 2 = 0 are x = ±1 and x = 2.


## The code interpreter built-in tool

In [44]:
loan_question = (
    "How much is the monthly payment, total payment, "
    "and total interest paid for a 30 year mortgage of $1M "
    "at a fixed rate of 6% with a 20% down payment?"
)

In [45]:
messages = [
    {"role": "system",     "content": system_prompt_content},
    {"role": "user",       "content": loan_question},
  ]

In [46]:
response = llama31(messages)
print(response)

import math

# Define the variables
principal = 800000  # $1M - $200,000 down payment
annual_interest_rate = 0.06
years = 30
months = years * 12

# Calculate the monthly interest rate
monthly_interest_rate = annual_interest_rate / 12

# Calculate the monthly payment
monthly_payment = principal * (monthly_interest_rate * (1 + monthly_interest_rate) ** months) / ((1 + monthly_interest_rate) ** months - 1)

# Calculate the total payment
total_payment = monthly_payment * months

# Calculate the total interest paid
total_interest_paid = total_payment - principal

print(f"Monthly payment: ${monthly_payment:.2f}")
print(f"Total payment: ${total_payment:.2f}")
print(f"Total interest paid: ${total_interest_paid:.2f}")


# Calculate loan payments

In [47]:
from utils import calculate_loan
monthly_payment, total_payment, total_interest_paid = calculate_loan(
    loan_amount = 1000000, 
    annual_interest_rate = 0.06, 
    loan_term = 30, 
    down_payment = 200000)

print(f"Monthly payment: ${(monthly_payment)}")
print(f"Total payment: ${(total_payment)}")
print(f"Total interest paid: ${(total_interest_paid)}")

Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705


## Generating the code in Java

In [48]:
messages = [
    {"role":    "system",     
     "content": system_prompt_content + "\nGenerate the code in Java."},
    {"role":    "user",       "content": loan_question},
  ]

In [49]:
response = llama31(messages)
print(response)

import java.util.Scanner;

public class MortgageCalculator {
    public static void main(String[] args) {
        // Define the variables
        double principal = 800000; // $1M
        double interestRate = 0.06; // 6%
        int years = 30;
        double downPayment = 0.2; // 20%
        double monthlyInterestRate = interestRate / 12;
        int numberOfPayments = years * 12;

        // Calculate the down payment
        double downPaymentAmount = principal * downPayment;
        principal -= downPaymentAmount;

        // Calculate the monthly payment
        double monthlyPayment = principal * (monthlyInterestRate * Math.pow(1 + monthlyInterestRate, numberOfPayments)) / (Math.pow(1 + monthlyInterestRate, numberOfPayments) - 1);

        // Calculate the total payment
        double totalPayment = monthlyPayment * numberOfPayments;

        // Calculate the total interest paid
        double totalInterestPaid = totalPayment - (principal * numberOfPayments);

        // Print t

### Reprompting Llama with Code Interpreter tool response

In [50]:
code_interpreter_tool_response="""
Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705
"""

In [51]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",      "content": loan_question},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": code_interpreter_tool_response}
  ]

In [52]:
response = llama31(messages)
print(response)

The monthly payment for a 30-year mortgage of $1M at a fixed rate of 6% with a 20% down payment is $4,796. The total payment over the life of the loan is $1,726,705, and the total interest paid is $926,705.


## Llama 3.1 custom tool calling

In [53]:
from utils import trending_songs, get_boiling_point

country = "US"
top_num = 5
top_songs = trending_songs(country, top_num)
print(f"Top {top_num} trending songs in {country}:")
print(top_songs)

Top 5 trending songs in US:
['Blinding Lights - The Weeknd', 'Levitating - Dua Lipa', 'Peaches - Justin Bieber', 'Save Your Tears - The Weeknd', 'Good 4 U - Olivia Rodrigo']


## Prompt Llama 3.1 for custom tool call

In [54]:
user_prompt = """
Answer the user's question by using the following functions if needed.
If none of the functions can be used, please say so.
Functions (in JSON format):
{
    "type": "function", "function": {
        "name": "get_boiling_point",
        "description": "Get the boiling point of a liquid",
        "parameters": {
            "type": "object", "properties": [
                {"liquid_name": {"type": "object", "description": "name of the liquid"}},
                {"celsius": {"type": "object", "description": "whether to use celsius"}}
            ], "required": ["liquid_name"]
        }
    }
}
{
    "type": "function", "function": {
        "name": "trending_songs",
        "description": "Returns the trending songs on a Music site",
        "parameters": {
            "type": "object", "properties": [
                {"country": {"type": "object", "description": "country to return trending songs for"}},
                {"n": {"type": "object", "description": "The number of songs to return"}}
            ], "required": ["country"]
        }
    }
}

Question: Can you check the top 5 trending songs in US?
"""

In [57]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt}
  ]

In [58]:
result = llama31(messages,405)
print(result)

{"type": "function", "name": "trending_songs", "parameters": {"country": "US", "n": "5"}}


### Calling the custom tool

In [59]:
custom_tools = {"trending_songs": trending_songs,
                "get_boiling_point": get_boiling_point}

In [60]:
res = json.loads(result)
function_name = res['name']
parameters = list(res['parameters'].values())
function_name, parameters

('trending_songs', ['US', '5'])

In [62]:
tool_result = custom_tools[function_name](*parameters)
tool_result

['Blinding Lights - The Weeknd',
 'Levitating - Dua Lipa',
 'Peaches - Justin Bieber',
 'Save Your Tears - The Weeknd',
 'Good 4 U - Olivia Rodrigo']

### Reprompting Llama with custom tool call result

In [63]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt},
    {"role": "assistant", "content": result},
    {"role": "ipython", "content": ','.join(tool_result)}
  ]

In [64]:
response = llama31(messages, 70)
print(response)

The top 5 trending songs in the US are:

1. Blinding Lights - The Weeknd
2. Levitating - Dua Lipa
3. Peaches - Justin Bieber
4. Save Your Tears - The Weeknd
5. Good 4 U - Olivia Rodrigo
